In [1]:
! pip install PyPDF2
! pip install chromadb
! pip install tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.5 MB/s et

In [2]:
from openai import OpenAI
import gradio as gr
import os
import PyPDF2
import chromadb
from dotenv import load_dotenv
import requests
from urllib.parse import urlparse
import re
import json
from datetime import datetime
import uuid

In [3]:
os.environ["OPENAI_API_KEY"] = "" #Add your OPENAI API Key
load_dotenv()
from openai import OpenAI
client = OpenAI(api_key="")

In [37]:
# Initialize Tavily client with error handling
os.environ["TAVILY_API_KEY"] = "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"
try:
    from tavily import TavilyClient
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    TAVILY_AVAILABLE = True
except ImportError:
    TAVILY_AVAILABLE = False
    print("⚠️  Tavily not installed. Web search functionality will be disabled.")
    print("💡 Run: pip install tavily-python")

In [21]:
# Initialize ChromaDB
chroma_client = chromadb.Client()
pdf_collection = chroma_client.create_collection(name="PhysicsPDFs")
memory_collection = chroma_client.create_collection(name="ConversationMemory")

InternalError: Collection [PhysicsPDFs] already exists

In [38]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [39]:
def chunk_text(text, chunk_size=1000):
    """Split text into chunks"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [40]:
def store_pdf_in_db(pdf_path):
    """Extract, chunk and store PDF in ChromaDB"""
    try:
        text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(text)

        # Clear existing data
        pdf_collection.delete(where={"source": {"$ne": ""}})  # Delete all documents

        # Add chunks to ChromaDB
        for i, chunk in enumerate(chunks):
            pdf_collection.add(
                documents=[chunk],
                metadatas=[{"source": pdf_path, "chunk_id": i}],
                ids=[f"chunk_{i}_{os.path.basename(pdf_path)}"]
            )
        return f"✅ Successfully loaded {len(chunks)} chunks from {pdf_path}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [41]:
def get_relevant_context(query, n_results=3):
    """Retrieve relevant context from ChromaDB"""
    try:
        results = pdf_collection.query(
            query_texts=[query],
            n_results=n_results
        )
        return "\n\n".join(results['documents'][0]) if results['documents'] else ""
    except Exception as e:
        print(f"ChromaDB query error: {e}")
        return ""

In [42]:
def store_conversation_memory(user_message, bot_response, conversation_context=""):
    """Store conversation in long-term memory"""
    try:
        # Create a memory entry
        memory_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()

        # Combine user message and bot response for semantic search
        memory_text = f"User: {user_message}\nBot: {bot_response}"

        if conversation_context:
            memory_text = f"Context: {conversation_context}\n{memory_text}"

        # Store in memory collection
        memory_collection.add(
            documents=[memory_text],
            metadatas=[{
                "type": "conversation",
                "timestamp": timestamp,
                "user_message": user_message,
                "bot_response": bot_response,
                "conversation_context": conversation_context
            }],
            ids=[memory_id]
        )

        return True
    except Exception as e:
        print(f"Memory storage error: {e}")
        return False

In [43]:
def get_relevant_memories(query, n_results=2):
    """Retrieve relevant past conversations from memory"""
    try:
        results = memory_collection.query(
            query_texts=[query],
            n_results=n_results
        )

        memories = []
        if results['documents']:
            for i, doc in enumerate(results['documents'][0]):
                metadata = results['metadatas'][0][i]
                memory_text = f"Previous conversation ({metadata['timestamp'][:10]}): {doc}"
                memories.append(memory_text)

        return "\n\n".join(memories) if memories else ""
    except Exception as e:
        print(f"Memory retrieval error: {e}")
        return ""

In [44]:
def get_conversation_summary(history):
    """Generate a summary of the current conversation context"""
    if not history:
        return ""

    # Take last few exchanges for context
    recent_history = history[-3:] if len(history) > 3 else history
    context_parts = []

    for exchange in recent_history:
        if isinstance(exchange, tuple) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")
        elif isinstance(exchange, list) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")

    return "\n".join(context_parts)

In [45]:
def is_pdf_context_relevant(pdf_context, query):
    """Check if PDF context is actually relevant to the query"""
    if not pdf_context or len(pdf_context.strip()) < 50:
        return False

    # Check for common irrelevant patterns in PDF context
    irrelevant_patterns = [
        "table of contents", "copyright", "abstract", "references",
        "appendix", "index", "acknowledg", "preface"
    ]

    pdf_lower = pdf_context.lower()
    query_lower = query.lower()

    # If PDF context contains many irrelevant sections, it's likely not useful
    irrelevant_count = sum(1 for pattern in irrelevant_patterns if pattern in pdf_lower)
    if irrelevant_count > 2:
        return False

    # Check if query terms actually appear in the context
    query_terms = query_lower.split()
    relevant_terms = sum(1 for term in query_terms if len(term) > 3 and term in pdf_lower)

    # If less than 30% of meaningful query terms are in context, consider it irrelevant
    meaningful_terms = [term for term in query_terms if len(term) > 3]
    if meaningful_terms and (relevant_terms / len(meaningful_terms)) < 0.3:
        return False

    return True

In [46]:
def search_with_tavily(query):
    """Search using Tavily and get content from first result"""
    if not TAVILY_AVAILABLE:
        return "Web search unavailable. Please install tavily-python: pip install tavily-python"

    try:
        # Search for the query
        search_response = tavily_client.search(
            query=query,
            search_depth="basic",
            max_results=3
        )

        if not search_response.get('results'):
            return "No search results found."

        # Get the first result
        first_result = search_response['results'][0]
        first_url = first_result.get('url', '')

        # Use Tavily's content if available
        content = first_result.get('content', '')

        if not content:
            content = f"Title: {first_result.get('title', 'N/A')}\nURL: {first_url}"

        # Check if it's a PDF
        if first_url.lower().endswith('.pdf'):
            return f"📄 PDF Source: {first_url}\n\nThis appears to be a PDF document. Please visit the link to view the PDF: {first_url}"

        return f"🔍 Search Result from: {first_url}\n\n{content}"

    except Exception as e:
        return f"❌ Search error: {str(e)}"

In [47]:
def chat_with_physics_bot(message, history):
    # Get relevant context from different sources
    pdf_context = get_relevant_context(message)
    memory_context = get_relevant_memories(message)
    conversation_context = get_conversation_summary(history)

    # Check if PDF context is actually relevant
    use_pdf = is_pdf_context_relevant(pdf_context, message)

    system_prompt = """You are a friendly Physics Teacher Bot with memory. Follow these rules:
    1. Use the provided context from PDFs, past conversations, and search results to answer questions accurately
    2. If the context doesn't fully answer the question, use your knowledge to provide a complete answer
    3. Reference past conversations when relevant to provide continuity
    4. Be educational and clear in your explanations
    5. Maintain context from the current conversation flow"""

    # Build context sections
    context_sections = []
    source_notes = []

    if use_pdf:
        context_sections.append(f"PDF Knowledge:\n{pdf_context}")
        source_notes.append("📚 PDF")

    if memory_context:
        context_sections.append(f"Past Conversations:\n{memory_context}")
        source_notes.append("💭 Memory")

    if conversation_context:
        context_sections.append(f"Current Conversation:\n{conversation_context}")

    # If no PDF context is relevant, use web search
    if not use_pdf:
        search_results = search_with_tavily(message)
        context_sections.append(f"Web Search:\n{search_results}")
        source_notes.append("🔍 Web")

    # Combine all contexts
    combined_context = "\n\n".join(context_sections) if context_sections else "No specific context available."
    source_note = " + ".join(source_notes) if source_notes else "General knowledge"

    user_content = f"""Available Context:
{combined_context}

Question: {message}

Please provide a helpful answer based on the available context above."""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ],
            temperature=0.2
        )

        bot_response = response.choices[0].message.content

        # Store this conversation in long-term memory
        store_conversation_memory(message, bot_response, conversation_context)

        return f"🔮 Sources: {source_note}\n\n{bot_response}"

    except Exception as e:
        return f"❌ Error generating response: {str(e)}"

In [48]:
def clear_memory():
    """Clear all conversation memory"""
    try:
        memory_collection.delete(where={"type": {"$ne": ""}})
        return "✅ Conversation memory cleared successfully!"
    except Exception as e:
        return f"❌ Error clearing memory: {str(e)}"

In [49]:
def export_memory():
    """Export conversation memory as JSON"""
    try:
        results = memory_collection.get()
        memories = []

        for i, doc in enumerate(results['documents']):
            memories.append({
                "content": doc,
                "metadata": results['metadatas'][i],
                "id": results['ids'][i]
            })

        filename = f"conversation_memory_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(memories, f, indent=2)

        return f"✅ Memory exported to {filename}"
    except Exception as e:
        return f"❌ Error exporting memory: {str(e)}"

In [50]:
def load_pdf_and_chat(pdf_file, message, history):
    # First store the PDF
    result_msg = store_pdf_in_db(pdf_file.name)

    # Then get response
    response = chat_with_physics_bot(message, history)

    return result_msg, response

In [51]:
def launch_app():
    with gr.Blocks() as demo:
        gr.Markdown("# ⚛️ Physics Teacher Chat with PDF RAG + Web Search + Memory")

        with gr.Row():
            with gr.Column():
                pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
                load_btn = gr.Button("Load PDF into Knowledge Base")
                load_status = gr.Textbox(label="Load Status", interactive=False)

                gr.Markdown("### Memory Management")
                with gr.Row():
                    clear_mem_btn = gr.Button("🧹 Clear Memory")
                    export_mem_btn = gr.Button("💾 Export Memory")
                mem_status = gr.Textbox(label="Memory Status", interactive=False)

                gr.Markdown("""
                ### How it works:
                1. **Upload a PDF** - Answers prioritize PDF content when relevant
                2. **Ask questions** - System uses PDF, web search, and past conversations
                3. **Long-term memory** - Remembers past conversations for context
                4. **Multiple sources** - Combines PDF 📚, Web 🔍, and Memory 💭
                """)

            with gr.Column():
                chat_interface = gr.ChatInterface(
                    fn=chat_with_physics_bot,
                    title="Chat with Physics Bot (with Memory)",
                    description="Ask questions about physics. I'll remember our past conversations and use multiple knowledge sources to help you!"
                )

        # Load PDF when button clicked
        load_btn.click(
            fn=lambda pdf: store_pdf_in_db(pdf.name) if pdf else "Please upload a PDF first",
            inputs=[pdf_input],
            outputs=[load_status]
        )

        # Memory management buttons
        clear_mem_btn.click(
            fn=clear_memory,
            outputs=[mem_status]
        )

        export_mem_btn.click(
            fn=export_memory,
            outputs=[mem_status]
        )

    demo.launch()

if __name__ == "__main__":
    # Check for required API keys
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY not found in environment variables")
        exit(1)

    if not os.getenv("TAVILY_API_KEY") and TAVILY_AVAILABLE:
        print("❌ TAVILY_API_KEY not found in environment variables")
        print("Please get your free API key from: https://app.tavily.com/")
        print("Continuing without web search functionality...")

    # Auto-load PDF if exists
    pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf')]
    if pdf_files:
        print(f"Found PDF: {pdf_files[0]}")
        store_pdf_in_db(pdf_files[0])

    print("🧠 Long-term memory enabled! The bot will remember past conversations.")
    launch_app()

🧠 Long-term memory enabled! The bot will remember past conversations.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5ac8a1588692791c91.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import requests
import os

def download_pdf_from_github(url, filename="PhysicsTeacher.pdf"):
    response = requests.get(url)
    response.raise_for_status()  # ensure no errors
    with open(filename, "wb") as f:
        f.write(response.content)
    return filename

In [ ]:
if __name__ == "__main__":
    # Correct GitHub raw URL
    github_pdf_url = "https://github.com/sumit1311singh/PhysicsTeacher-CurrentElectricity-Bot/raw/main/Current_Electricity.pdf"

    # Download PDF
    pdf_file = download_pdf_from_github(github_pdf_url, "Physics-PDFs")
    print(f"Downloaded PDF: {pdf_file}")

    # Store in DB
    store_pdf_in_db(pdf_file)

    # Launch app
    launch_app()

Downloaded PDF: Physics-PDFs


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 92.4MiB/s]
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5eb52757e590cadfe9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
